## Now that you have built a dataframe of the postal code of each neighborhood 
## along with the borough name and neighborhood name, 
## in order to utilize the Foursquare location data, 
## we need to get the latitude and the longitude coordinates of each neighborhood.


### In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

# Part 0: Extracting the table again

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')
#print(soup.prettify())


In [2]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)
    

In [3]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)


# Part 1: Pre-processing - Installing geocoder

In [4]:
!pip install geocoder
"""
Collecting geocoder
  Downloading https://files.pythonhosted.org/packages/4f/6b/13166c909ad2f2d76b929a4227c952630ebaf0d729f6317eb09cbceccbab/geocoder-1.38.1-py2.py3-none-any.whl (98kB)
Requirement already satisfied: click in c:\programdata\anaconda3\lib\site-packages (from geocoder) (6.7)
Requirement already satisfied: six in c:\programdata\anaconda3\lib\site-packages (from geocoder) (1.11.0)
Collecting ratelim (from geocoder)
  Downloading https://files.pythonhosted.org/packages/f2/98/7e6d147fd16a10a5f821db6e25f192265d6ecca3d82957a4fdd592cad49c/ratelim-0.1.6-py2.py3-none-any.whl
Requirement already satisfied: requests in c:\programdata\anaconda3\lib\site-packages (from geocoder) (2.19.1)
Requirement already satisfied: future in c:\programdata\anaconda3\lib\site-packages (from geocoder) (0.16.0)
Requirement already satisfied: decorator in c:\programdata\anaconda3\lib\site-packages (from ratelim->geocoder) (4.3.0)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (3.0.4)
Requirement already satisfied: idna<2.8,>=2.5 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (2.7)
Requirement already satisfied: urllib3<1.24,>=1.21.1 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (1.23)
Requirement already satisfied: certifi>=2017.4.17 in c:\programdata\anaconda3\lib\site-packages (from requests->geocoder) (2018.8.24)
Installing collected packages: ratelim, geocoder
Successfully installed geocoder-1.38.1 ratelim-0.1.6
"""

print('geocoder has been installed before.')
import geocoder
print('geocoder has been successfully imported.')

    100% |████████████████████████████████| 102kB 3.3MB/s ta 0:00:01
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
  The script geocode is installed in '/gpfs/fs01/user/sdab-c7748cfedf25ba-34abd17332a4/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
geocoder has been installed before.
geocoder has been successfully imported.


# Part 2: Processing - Extracting Latitude and Longtitude for each Postal code in Toronto

In [6]:
Postcode_u

['M9A',
 'M4H',
 'M1B',
 'M9W',
 'M9L',
 'M4Y',
 'M9N',
 'M3J',
 'M2H',
 'M2J',
 'M5S',
 'M1T',
 'M6L',
 'M1W',
 'M3H',
 'M9C',
 'M3K',
 'M5J',
 'M5K',
 'M4N',
 'M4T',
 'M5T',
 'M6A',
 'M4K',
 'M4L',
 'M4X',
 'M5X',
 'M6S',
 'M4B',
 'M2K',
 'M2L',
 'M9M',
 'M1X',
 'M1M',
 'M6H',
 'M4W',
 'M9R',
 'M3L',
 'M6K',
 'M1C',
 'M9V',
 'M6P',
 'M1R',
 'M8W',
 'M2R',
 'M5A',
 'M3A',
 'M1J',
 'M6M',
 'M1V',
 'M1L',
 'M1G',
 'M1K',
 'M5L',
 'M9P',
 'M4V',
 'M4C',
 'M8X',
 'M6R',
 'M5V',
 'M3C',
 'M4A',
 'M5M',
 'M1N',
 'M5B',
 'M9B',
 'M6J',
 'M8Z',
 'M5R',
 'M2M',
 'M4E',
 'M5P',
 'M6B',
 'M5E',
 'M8V',
 'M1E',
 'M5C',
 'M1H',
 'M8Y',
 'M5H',
 'M1S',
 'M1P',
 'M4J',
 'M4G']

In [8]:
latitude = []
longitude = []
for elem in Postcode_u:
# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while (lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(elem))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    print(elem, 'is RECEIVED.')
    # print(lat_lng_coords[0])
    # print(lat_lng_coords[1])
    

KeyboardInterrupt: 

# Part 3: Post-processing - Creating Pandas Dataframe

In [34]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u,
              'Latitude': latitude, 'Longitude':longitude}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_base.csv')
df_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
1,M4G,East York,Leaside,43.709060,-79.363452
2,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
3,M1W,Scarborough,Steeles West,43.799525,-79.318389
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,M9P,Etobicoke,Westmount,43.696319,-79.532242
6,M1G,Scarborough,Woburn,43.770992,-79.216917
7,M4J,East York,East Toronto,43.685347,-79.338106
8,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Mimico NE, Old M...",43.636258,-79.498509
9,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
